In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from datetime import datetime

import statsmodels.api as sm
from statsmodels.tsa.stattools import grangercausalitytests

In [10]:
# mpl.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['font.size'] = 16

# Try granger causality

In [11]:
df = pd.read_csv("aggregated_individual_data/0_aggregated.csv", index_col=0, header=0, parse_dates=["date"])
# df["date"] = df["date"]
df= df.iloc[27:-1]
df['days'] = (df['date'] - df['date'].min()).dt.days.astype(int)
df.drop(["date","weekday", 'appCat.weather'], axis=1,inplace=True)
# df.drop("Unnamed: 0", axis=1, inplace=True)
df

,mood,circumplex.arousal,circumplex.valence,activity,screen,call,sms,appCat.builtin,appCat.communication,appCat.entertainment,...,appCat.other,appCat.social,appCat.travel,appCat.unknown,appCat.utilities,mood_std,circumplex.arousal_std,circumplex.valence_std,activity_std,days
29,6.60,-0.40,0.60,0.056918,15018.480001,2.0,0.0,552.201,11853.339,142.241,...,53.763,2107.146,0.000,0.000,49.718,0.0,0.0,0.0,0.077915,0
30,6.80,-0.80,0.80,0.094033,13490.729000,1.0,0.0,1860.898,5341.657,774.354,...,47.789,1993.922,63.874,0.000,88.680,0.0,0.0,0.0,0.138848,1
31,7.80,-0.20,1.00,0.099432,13293.248001,1.0,0.0,946.190,2728.260,469.274,...,31.482,396.672,128.890,0.000,305.796,0.0,0.0,0.0,0.115524,2
32,7.25,-0.25,0.75,0.074816,8343.518000,2.0,2.0,897.086,4984.724,727.492,...,34.202,2237.742,0.000,0.000,121.035,0.0,0.0,0.0,0.116477,3
33,7.60,-0.25,1.00,0.102041,3832.990999,2.0,0.0,3442.232,2004.347,74.880,...,58.049,428.972,68.278,0.000,210.882,0.0,0.0,0.0,0.162982,4
34,7.40,0.20,1.00,0.091930,15490.269999,2.0,0.0,2212.954,11360.913,1101.017,...,63.160,1422.548,0.000,0.000,423.215,0.0,0.0,0.0,0.106167,5
35,7.60,0.20,1.00,0.045986,14535.339999,1.0,0.0,1130.842,8928.472,347.427,...,42.398,526.598,0.000,0.000,246.944,0.0,0.0,0.0,0.067673,6
36,7.60,-0.40,1.00,0.050903,22086.478000,1.0,0.0,1466.271,9692.133,4464.975,...,31.933,2688.536,51.184,0.000,716.197,0.0,0.0,0.0,0.085740,7
37,7.20,-0.20,0.80,0.109061,13973.942000,2.0,1.0,817.613,7279.144,650.832,...,68.847,2227.694,63.316,0.000,347.974,0.0,0.0,0.0,0.126012,8
38,7.60,-0.20,1.00,0.214249,15482.018999,3.0,4.0,864.519,2567.857,125.556,...,46.205,2368.377,470.773,3.148,315.763,0.0,0.0,0.0,0.316180,9


In [12]:
df.to_csv('test.csv')

In [13]:
def granger_causality_matrix(data, variables, test="ssr_chi2test", verbose=False, maxlag=7):
    df = pd.DataFrame(np.zeros((len(variables),len(variables))), columns=variables, index=variables)
    for c in df.columns:
        for r in df.index:
            test_result = grangercausalitytests(data[[r, c]], maxlag=maxlag, verbose=False)
            p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
            if verbose: print(f'Y = {r}, X = {c}, P Values = {p_values}')
            min_p_value = np.min(p_values)
            df.loc[r, c] = min_p_value
    df.columns = [var + '_x' for var in variables]
    df.index = [var + '_y' for var in variables]
    return df

In [17]:
# Use your granger_causality_matrix function on the days column
variables = ['mood', 'circumplex.arousal', 'circumplex.valence', 'activity', 'screen', 'call', 'sms', 'appCat.builtin', 'appCat.communication', 'appCat.entertainment', 'appCat.finance', 'appCat.game', 'appCat.office', 'appCat.other', 'appCat.social', 'appCat.travel', 'appCat.unknown', 'appCat.utilities']
granger_matrix = granger_causality_matrix(df[['days'] + variables], variables)
granger_matrix

ValueError: Insufficient observations. Maximum allowable lag is 4

In [16]:
plt.figure(figsize=(12,12))
sns.heatmap(granger_matrix, cmap="Spectral_r")
plt.show()

NameError: name 'granger_matrix' is not defined

<Figure size 1200x1200 with 0 Axes>